<a href="https://colab.research.google.com/github/Chpppy/Colab/blob/main/notebooks/PixelCopter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -r https://raw.githubusercontent.com/huggingface/deep-rl-class/main/notebooks/unit4/requirements-unit4.txt

  Cloning https://github.com/ntasfi/PyGame-Learning-Environment.git to /tmp/pip-req-build-k0cwbrnu
  Running command git clone --filter=blob:none --quiet https://github.com/ntasfi/PyGame-Learning-Environment.git /tmp/pip-req-build-k0cwbrnu
  Resolved https://github.com/ntasfi/PyGame-Learning-Environment.git to commit 3dbe79dc0c35559bb441b9359948aabf9bb3d331
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/simoninithomas/gym-games to /tmp/pip-req-build-nxecbclc
  Running command git clone --filter=blob:none --quiet https://github.com/simoninithomas/gym-games /tmp/pip-req-build-nxecbclc
  Resolved https://github.com/simoninithomas/gym-games to commit f31695e4ba028400628dc054ee8a436f28193f0b
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.9/757.9 kB 8.7 MB/s eta 0:00:00
  Created wheel for ple: filename=ple-0.0.1-py3-none-any.whl size=50772 sha256=20faf4608c62a6a8f1f325393c6df2476bc2a3c1b36cb257d3e2c1ddf3815b6d
  Stored in dire

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical

import gym
import gym_pygame

import numpy as np
from collections import deque

In [3]:
env_id = "Pixelcopter-PLE-v0"
env = gym.make(env_id)
state_size = env.observation_space.shape[0]
action_size = env.action_space.n

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

couldn't import doomish
Couldn't import doom
cpu


/usr/local/lib/python3.11/dist-packages/pygame/pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists
/usr/local/lib/python3.11/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/local/lib/python3.11/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.cloud')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-pa

In [4]:
print("_____OBSERVATION SPACE_____ \n")
print("The State Space is: ", state_size)
print("Sample observation", env.observation_space.sample()) # Get a random observation

print("\n _____ACTION SPACE_____ \n")
print("The Action Space is: ", action_size)
print("Action Space Sample", env.action_space.sample()) # Take a random action

_____OBSERVATION SPACE_____ 

The State Space is:  7
Sample observation [ 0.5959286   0.94595546 -1.1859205   1.6790326   0.6182842  -0.7321073
  1.2180805 ]

 _____ACTION SPACE_____ 

The Action Space is:  2
Action Space Sample 1


In [5]:
class Policy(nn.Module):
    def __init__(self, state_size, hidden_size, action_size):
        super().__init__()
        self.fc1 = nn.Linear(state_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size * 2)
        self.fc3 = nn.Linear(hidden_size * 2, action_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.softmax(x, dim=1)

    def act(self, state):
        tensor_state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.forward(tensor_state).cpu()
        m = Categorical(probs)
        action = m.sample()
        return action.item(), m.log_prob(action)

In [6]:
param = {
    "state_size" : state_size,
    "hidden_size" : 64,
    "action_size" : action_size,
    "total_training_episode" : 50_000,
    "max_step" : 1000,
    "lr" : 5e-5,
    "gamma" : 0.99
}

In [7]:
PixelPolicy = Policy(param["state_size"], param["hidden_size"], param["action_size"]).to(device)
optimizer = torch.optim.Adam(PixelPolicy.parameters(), param["lr"])
scheduler = optimizer.lr_scheduler.LinearLR(optimizer, start_factor=1, end_factor=1e-1, total_iters=param["total_training_episode"])

In [8]:
def reinforce(env, optimizer, max_step, total_training_episode, gamma, print_time):
    scores = deque(maxlen=100)
    score = []
    for episode in range(1, total_training_episode + 1):
        rewards = []
        log_probs = []
        state = env.reset()
        for i in range(max_step):
            action, log_prob = PixelPolicy.act(state)
            log_probs.append(log_prob)
            state, reward, done, _ = env.step(action)
            rewards.append(reward)
            if done:
                break

        scores.append(sum(rewards))
        score.append(sum(rewards))
        cumulative_reward = deque(maxlen=max_step + 1)
        cumulative_reward.append(0)

        for i in range(len(rewards))[::-1]:
            cumulative_reward.appendleft(rewards[i] + gamma * cumulative_reward[0])
            if(i != 0):
              log_probs[i-1] = log_probs[i] + log_probs[i - 1]

        cumulative_reward = torch.tensor(cumulative_reward)
        eps = np.finfo(np.float32).eps.item()

        # cumulative_reward = (cumulative_reward - cumulative_reward.mean()) / (cumulative_reward.std() + eps)

        policy_loss = []
        for log_prob, disc_return in zip(log_probs, cumulative_reward):
            policy_loss.append(-log_prob * disc_return)
        policy_loss = torch.cat(policy_loss).sum() / len(log_probs)

        optimizer.zero_grad()
        policy_loss.backward()
        optimizer.step()

        scheduler.step()
        if episode % print_time == 0:
            print('Episode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores)))

    return score

In [ ]:
score = reinforce(env, optimizer, param["max_step"], param["total_training_episode"], param["gamma"], 200)

/usr/local/lib/python3.11/dist-packages/gym/utils/passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
/usr/local/lib/python3.11/dist-packages/gym/utils/passive_env_checker.py:190: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `return_info` to return information from the environment resetting.
  logger.warn(
/usr/local/lib/python3.11/dist-packages/gym/utils/passive_env_checker.py:195: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(
/usr/local/lib/python3.11/dist-packages/gym/utils/passive_env_checker.py:141: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
/usr

Episode 200	Average Score: -4.31
Episode 400	Average Score: 0.58
Episode 600	Average Score: 1.83
Episode 800	Average Score: 2.89
Episode 1000	Average Score: 3.13
Episode 1200	Average Score: 3.50
Episode 1400	Average Score: 5.08
Episode 1600	Average Score: 3.99
Episode 1800	Average Score: 3.71
Episode 2000	Average Score: 4.89
Episode 2200	Average Score: 4.06
Episode 2400	Average Score: 4.97
Episode 2600	Average Score: 3.81
Episode 2800	Average Score: 4.27
Episode 3000	Average Score: 4.70
Episode 3200	Average Score: 4.95
Episode 3400	Average Score: 2.84
Episode 3600	Average Score: 6.44
Episode 3800	Average Score: 5.69
Episode 4000	Average Score: 6.96
Episode 4200	Average Score: 6.13
Episode 4400	Average Score: 4.80
Episode 4600	Average Score: 6.88
Episode 4800	Average Score: 5.15
Episode 5000	Average Score: 7.46
Episode 5200	Average Score: 4.69
Episode 5400	Average Score: 6.99
Episode 5600	Average Score: 6.90
Episode 5800	Average Score: 7.36
Episode 6000	Average Score: 5.62
Episode 6200	

In [ ]:
torch.save(PixelPolicy.state_dict(), "Pixelcopter-PLE.pth")